# CH9 Up and Running with TensorFlow

Tensorflow의 기본 원리는 간단하다: 먼저 파이썬으로 밑의 그림과 같은 계산을 정의한다. 그후 텐서플로우가 이 그래프를 가지고 최적화된 C++코드를 통해 이를 연산한다.

<img src="ch9_1.png" alt="Drawing" style="width: 700px;"/>

더 중요한 특징으로는, 위와 같은 그래프는 몇개의 chunk로 나눠 CPU나 GPU에서 병렬연산을 할 수 있는 것이다. 따라서 엄청나게 큰 신경망을 쪼개 몇백대의 서버로 나눠 연산할 수 있고, 이는 연산시간을 크게 줄일 수 있다. 또한 텐서플로우는 몇백만개의 파라메터를 가진 몇십억개의 인스턴스 네트워크 또한 훈련시킬수 있다.

<img src="ch9_2.png" alt="Drawing" style="width: 700px;"/>

다음은 텐서플로우의 특징들이다.

- 윈도우,리눅스, 맥OS를 포함한 모바일 디바이스(iOS, Android)에서 구동 가능
- TF.Learn이라는 심플한 파이썬 API 제공(scikit-learn과 같이 사용 가능)
- Keras나 Pretty Tensor같은 하이레벨 API는 텐서플로우 위에 지어짐
- 메인 파이썬 API는 엄청나게 flexible하다.
- 엄청 효율적인 C++ implementation을 포함한다.
- 몇몇 advanced된 최적화 노드(비용 최소화시키는 파라메터 찾기 위한)을 가진다. 즉 텐서플로우는 자동적으로 너가 정의한 함수의 gradients를 computing하는 것에 관여한다.
- 텐서보드라는 시각화 툴을 제공한다
- 텐서플로우 그래프를 위한 cloud service도 제공한다.
- 커뮤니티가 좋다!!

<img src="ch9_3.png" alt="Drawing" style="width: 700px;"/>

## Installation

Documentation 참고해서 깔자. Anaconda env에 깔아야 주피터에서 쓸 수 있다. Virtualenv 만들어서 깔면 귀찮아짐..(안에 주피터 또깔아야함)

In [1]:
import tensorflow as tf
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y") 
f = x*x*y + y + 2

위 코드는 바로 연산을 하지 않는다. 이는 연산 그래프를 만든다. 사실, 변수조차 아직 initalize되지 않았다. 이를 evaluate하기 위해서는 텐서플로우 session을 열어서 변수들을 initialize하고 f를 evaluate해야한다. 텐서플로우 세션은 연산을 CPU나 GPU에 올려서 연산시키고, 변수값을 hold한다. 

In [2]:
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)
sess.close()

42


계속 sess.run하는게 귀찮을 수 있다. 물론 다른 좋은 방법이 있다.

In [3]:
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()

with 블럭 안에 세션은 default 세션으로 세팅된다. 
 - `x.initializer.run() <-> tf.get_default_session().run(x.initializer)`
 - `f.eval <-> tf.get_default_session().run(f)`
 

또한 각각의 변수마다 initializer를 돌리기 보다는, global_variables_initializer()를 사용할 수도 있다. *주의할 점은, 이는 initialization을 시행하는것이 **아니라** 실행시 모든 변수를 initialize 하는 노드를 하나 생성하는 것이다.*

In [4]:
init = tf.global_variables_initializer() # prepare an init node

with tf.Session() as sess:
    init.run() # actually initialize all the variables 
    result = f.eval()

주피터나 파이썬 쉘 안에 `InterativeSession`을 만들수도 있다. 이것이 그냥 Session과 다른 점은, `InterativeSession`이 만들어질 때는 이는 자기 자신을 기본 세션으로 세팅한다. 그래서 with block이 필요가 없다.(물론 이렇게 하면 세션을 닫아줘야 한다)

In [5]:
init = tf.global_variables_initializer()
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

42


즉 텐서플로우는 처음에 연산 그래프를 만들고, 그다음 이를 실행시킨다. 

## Managing Graphs

니가 만든 노드는 자동적으로 default 그래프에 추가된다:

In [6]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

거의 대부분의 케이스에 괜찮지만, 여러개의 독립적 그래프를 관리하고 싶을때가 있다. 

In [7]:
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)

x2.graph is graph

True

In [8]:
x2.graph is tf.get_default_graph()

False

    * 주피터에서는 같은 커멘드를 여러번 돌릴때가 있다. 그 결과로 default graph가 많은 복제된 (같은)노드를 가지고 있을 수 있다. 주피터 커널을 다시 시작할수도 있지만, 더 좋은 방법은 default 그래프를 리셋하는것이다( tf.reset_default_graph() )

## Lifecycle of a Node Value

텐서플로우의 노드 연산

In [9]:
w = tf.constant(3)
x = w + 2
y = x + 5
z = x * 3

In [10]:
with tf.Session() as sess:
    print(y.eval())
    print(z.eval()) # x를 먼저 연산함

10
15


주의할 점은, 위 코드에서 x와 w를 두번 연산했다는 것이다( 이전 연산 결과를 재사용하지 않음).

세션 그래프 연산을 통해 유지되는 변수 값을 제외한 모든 노드값은 그래프가 연산되는 사이에 drop된다(ch12에서 더 깊게 본다). 변수는 initializer가 실행될때 값이 있게 되고, 세션이 닫힐때 끝난다.

y, z를 효율적으로 연산하려면 다음과 같이 함께(한 그래프에서) 연산하면 된다. 



In [11]:
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
15


## Linear Regression with Tensorflow

텐서플로우의 input과 ouput은 다차원 어레이로, 텐서라고 불린다. Numpy array처럼 텐서는 type과 shape이 존재한다. 실제로 파이썬 API에서 텐서는 numpy ndarray로 표현된다. 주로 float를 담지만, string을 담을수도 있다.

In [12]:
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m, 1)), housing.data] # col에 extra bias 붙이기

X = tf.constant(housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
XT = tf.transpose(X) 
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(XT, X)), XT), y)
# matmul: matrix multiplication

with tf.Session() as sess:
    theta_value = theta.eval()

<img src="ch9_theta.png" alt="Drawing" style="width: 500px;"/>

텐서플로우에서 위를 실행하면 만약 GPU 서포트가 있다면 자동으로 GPU에서 연산해준다.

## Implementing Gradient Descent

Batch Gradient Descent를 이용해보자.

    *Gradient Descent쓸때 먼저 normalize 하는거 잊지말자.

In [13]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

### Manually Computing the Gradients

In [14]:
n_epochs = 1000
learning_rate = 0.01

X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")
y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
# reduce_mean: Computes the mean of elements across dimensions of a tensor.

gradients = 2/m * tf.matmul(tf.transpose(X), error)
training_op = tf.assign(theta, theta - learning_rate * gradients)
# assign: 새로운 노드를 만든다. theta = theta - r * MSE(theta)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE =", mse.eval())
        sess.run(training_op)
    
    best_theta = theta.eval()

Epoch 0 MSE = 2.75443
Epoch 100 MSE = 0.632222
Epoch 200 MSE = 0.57278
Epoch 300 MSE = 0.558501
Epoch 400 MSE = 0.549069
Epoch 500 MSE = 0.542288
Epoch 600 MSE = 0.537379
Epoch 700 MSE = 0.533822
Epoch 800 MSE = 0.531243
Epoch 900 MSE = 0.529371


### Using autodiff

선형회귀의 경우 위와가이 직접 다 코드를 써도 상대적으로 어렵지 않다. 하지만 딥 뉴럴넷을 짤때 이는 굉장히 힘들 것이다. 따라서 `symbolic differentiation`을 사용할 수 있을 것이다. 이는 자동적으로 편미분식을 구하지만 그리 효율적이지는 않다.

왜냐면 f(x)= exp(exp(exp(x))) 이라 하자. 이 도함수는 f′(x) = exp(x) × exp(exp(x)) × exp(exp(exp(x))) 이다. 니가 만약 f(x)와 f'(x)를 따로 작성한다면 그리 효율적이지 않고, 대신 exp(x) -> exp(exp(x)) -> exp(exp(exp(x))) 를 계산하는 함수를 만들어 위 세개를 다 계산하는게 낫다. 

In [15]:
def my_func(a, b):

    z = 0
    for i in range(100):
        z = a * np.cos(z + i) + z * np.sin(b - i)
    return z

위 식의 편미분 도함수를 찾아낼수 있는가? 거의 힘들다. 다행히 텐서플로우의 autodiff 기능은 자동적, 효율적으로 gradients를 계산한다. 

In [16]:
gradients = tf.gradients(mse, [theta])[0]

위 함수는 먼저 mse와 같은 에러 비용함수에 대한 각각의 변수의 gradients를 계산한다. (위에서는 theta) 따라서 gradients 노드는 theta에 대한 MSE gradient vector를 계산한다. 텐서플로우는 reverse-mode autodiff을 이용한다. 이는 많은 인풋과 적은 아웃풋이 있을때 좋으며, 아웃풋에 대한 paritial derivatives를 계산한다. 궁금하면 Appendix D를 찾아봐라.

<img src="ch9_4.png" alt="Drawing" style="width: 700px;"/>

### Using an Optimizer

텐서플로우는 optimizer들도 많이 제공한다. 단지 밑과 같이 쓰면 된다.

In [17]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate) 
training_op = optimizer.minimize(mse)

다른 타입을 쓰려면 밑과 같이 한다.

In [19]:
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)

## Feeding Data to the Training Algorithm

위 코드로 mini batch GD 만들어 보자. 그럼 먼저 X, y 바꿔야 한다. 가장 쉬운 방법은 placeholder node를 쓰는 것이다. 이들은 실제 계산을 하지 않고, 실행시 아웃풋을 계산한다. 이는 보통 텐서플로우가 돌아가는 도중 훈련 데이터를 넘겨줄때 사용한다.

In [20]:
A = tf.placeholder(tf.float32, shape=(None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict={A: [[1, 2, 3]]})
    B_val_2 = B.eval(feed_dict={A: [[4, 5, 6], [7, 8, 9]]})

print(B_val_1)

[[ 6.  7.  8.]]


In [21]:
print(B_val_2)

[[  9.  10.  11.]
 [ 12.  13.  14.]]


In [24]:
X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X") 
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")

batch_size = 100
n_batches = int(np.ceil(m / batch_size)) # np.ceil: 올림

In [25]:
def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()


## Saving and Restoring Models

모델을 훈련 시키면, 그것의 파라메터를 디스크에 저장시켜 다시 꺼내쓸 수 있다. 또한 훈련 도중 체크포인트를 저장시켜 그 포인트부터 다시 훈련시킬수도 있다.

밑은 Saver node를 추가시킨 다음, 실행시 save() 메소드를 사용해 저장시키면 된다.

In [ ]:
#n_epochs = 1000                                                                       # not shown in the book
#learning_rate = 0.01                                                                  # not shown

#X = tf.constant(scaled_housing_data_plus_bias, dtype=tf.float32, name="X")            # not shown
#y = tf.constant(housing.target.reshape(-1, 1), dtype=tf.float32, name="y")            # not shown
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
#y_pred = tf.matmul(X, theta, name="predictions")                                      # not shown
#error = y_pred - y                                                                    # not shown
#mse = tf.reduce_mean(tf.square(error), name="mse")                                    # not shown
#optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)            # not shown
#training_op = optimizer.minimize(mse)                                                 # not shown

init = tf.global_variables_initializer()
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        if epoch % 100 == 0: # checkpoint every 100 epochs
            #print("Epoch", epoch, "MSE =", mse.eval())                                # not shown
            save_path = saver.save(sess, "/tmp/my_model.ckpt")
        sess.run(training_op)
    
    best_theta = theta.eval()
    save_path = saver.save(sess, "/tmp/my_model_final.ckpt")

그 모델을 다시 불러오는것 또한 쉽다. 

In [ ]:
with tf.Session() as sess:
    saver.restore(sess, "/tmp/my_model_final.ckpt")
    [...]

save or restore할 변수들을 정해줄 수도 있다.

In [26]:
saver = tf.train.Saver({"weights": theta}) # weights 라는 이름으로 theta 저장

## Visualizing the Graph and Training Curves Using TensorBoard

텐서보드를 이용하려면 먼저 프로그램 안에 그래프 정의를 쓰고 training stats들을 log directory(텐서보드가 읽을)에 쓰는 것이다. 프로그램을 돌릴때마다 다른 로그 디렉토리가 필요하다. 아니면 텐서보드가 stat들을 통합해 시각화가 안좋아 진다. 가장 좋은 방법은 timestamp를 붙이는 것이다. 다음 코드를 프로그램 시작시 붙여준다.

In [27]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)

그 다음 다음 코드를 construction phase 마지막에 붙여준다.

In [28]:
mse_summary = tf.summary.scalar('MSE', mse) 
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())

다음 execution phase에서 mse_summary 노드를 다음과 같이 붙여준다. 

In [ ]:
#with tf.Session() as sess:                                                        # not shown in the book
#    sess.run(init)                                                                # not shown

#    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()      

마지막으로, 프로그램 마지막에 filewriter 를 닫아주면 된다.

In [ ]:
file_writer.close()

In [1]:
from datetime import datetime

now = datetime.utcnow().strftime("%Y%m%d%H%M%S")
root_logdir = "tf_logs"
logdir = "{}/run-{}/".format(root_logdir, now)
n_epochs = 1000
learning_rate = 0.01

X = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="y")
theta = tf.Variable(tf.random_uniform([n + 1, 1], -1.0, 1.0, seed=42), name="theta")
y_pred = tf.matmul(X, theta, name="predictions")
error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name="mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

mse_summary = tf.summary.scalar('MSE', mse)
file_writer = tf.summary.FileWriter(logdir, tf.get_default_graph())
n_epochs = 10
batch_size = 100
n_batches = int(np.ceil(m / batch_size))

with tf.Session() as sess:                                                        # not shown in the book
    sess.run(init)                                                                # not shown

    for epoch in range(n_epochs):                                                 # not shown
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            if batch_index % 10 == 0:
                summary_str = mse_summary.eval(feed_dict={X: X_batch, y: y_batch})
                step = epoch * n_batches + batch_index
                file_writer.add_summary(summary_str, step)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})

    best_theta = theta.eval()       
    
file_writer.close()

NameError: name 'tf' is not defined

위같이 다 돌린다음 working directory에 가서, shell 에서 ls -l tf_logs/run*  를 치면 다음과 같이 나온다. (두번 돌려서 두개)

<img src="ch9_log.png" alt="Drawing" style="width: 700px;"/>

그다음 텐서보드 서버를 켜보자. log file 있는 곳으로 가서 tensorflow activate 한다.

In [ ]:
$ source env/bin/activate 
$ tensorboard --logdir tf_logs/ 
Starting TensorBoard on port 6006
(You can navigate to http://0.0.0.0:6006)

<img src="ch9_5.png" alt="Drawing" style="width: 700px;"/>

<img src="ch9_6.png" alt="Drawing" style="width: 700px;"/>

주피터 안에서 위와같은 그래프 바로 그리려면, show_graph()를 사용할수도 있다. (책 236page 참고)

## Name Scopes

뉴럴넷과 같은 복잡한 모델들 사용할때는, 그래프가 너무 복잡해질수 있다. 이를 피하기 위해 name scope라는 것을 만들어 연결된 노드들을 그룹화한다. 

In [32]:
with tf.name_scope("loss") as scope:
    error = y_pred - y
    mse = tf.reduce_mean(tf.square(error), name="mse")

In [33]:
print(error.op.name)

loss/sub


In [34]:
print(mse.op.name)

loss/mse


<img src="ch9_7.png" alt="Drawing" style="width: 700px;"/>

## Modularity

만약 너가 두개의 ReLU(Rectified linear units) 아웃풋을 더하는 그래프를 그리고 싶다고 하자. ReLU는 인풋의 선형함수를 계산한다. 다음과 같은 코드와 같다.

In [37]:
n_features = 3 
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")

w1 = tf.Variable(tf.random_normal((n_features, 1)), name="weights1") 
w2 = tf.Variable(tf.random_normal((n_features, 1)), name="weights2")
b1 = tf.Variable(0.0, name="bias1")
b2 = tf.Variable(0.0, name="bias2")

z1 = tf.add(tf.matmul(X, w1), b1, name="z1") 
z2 = tf.add(tf.matmul(X, w2), b2, name="z2")

relu1 = tf.maximum(z1, 0., name="relu1") 
relu2 = tf.maximum(z1, 0., name="relu2")

output = tf.add(relu1, relu2, name="output")

하지만 위 코드는 굉장히 반복적이다. 더 쓴다면 에러를 만들수도 있다. 다음과 같이 이를 깔끔하게 쓸 수 있다.

In [38]:
def relu(X):
    w_shape = (int(X.get_shape()[1]), 1)
    w = tf.Variable(tf.random_normal(w_shape), name="weights")
    b = tf.Variable(0.0, name="bias")
    z = tf.add(tf.matmul(X, w), b, name="z")
    return tf.maximum(z, 0., name="relu")

n_features = 3
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X) for i in range(5)]
output = tf.add_n(relus, name="output")

텐서플로우는 노드를 새로 만들때 그것의 이름이 이미 있는 것인지 체크한다. 만약 그렇다면 _1, _2와 같이 언더바를 붙인다.

<img src="ch9_8.png" alt="Drawing" style="width: 700px;"/>

name scope를 이용하면, 그래프를 훨씬 깔끔하게 그릴 수 있다.

In [39]:
def relu(X):
    with tf.name_scope("relu"):
    [...]

IndentationError: expected an indented block (<ipython-input-39-9f23f91341f6>, line 3)

<img src="ch9_9.png" alt="Drawing" style="width: 700px;"/>

## Sharing Variables

그래프 상의 다양한 component 사이에 변수를 공유하고 싶다면, 먼저 이를 만든 후 다른 함수에 파라메터로 넘기는 것이다.

In [ ]:
def relu(X, threshold):
    with tf.name_scope("relu"):
    [...]
    return tf.maximum(z, threshold, name="max") # threshold를 공유하고 싶다면

threshold = tf.Variable(0.0, name="threshold") # threshold를 만들고
X = tf.placeholder(tf.float32, shape=(None, n_features), name="X")
relus = [relu(X, threshold) for i in range(5)]  #Relu에 pass
output = tf.add_n(relus, name="output")

하지만 만약 공유해야 할 변수가 많을때 이를 모두 파라메터로 넘기는 것은 힘들 것이다. 많은 방법이 있지만, 텐서플로우는 다른 옵션을 준다. 아이디어는 get_variable() 함수를 이용해 공용 변수를 만드는 것이다(아직 변수가 없다면, 만약 있다면 재사용). 재사용하거나 새로 만들어진 것들은 현재 variable_scope()에 의해 컨트롤 된다. 

In [41]:
with tf.variable_scope("relu"):
    threshold = tf.get_variable("threshold", shape=(), 
                            initializer=tf.constant_initializer(0.0))

만약 재사용하고 싶다면, 다음과 같이 한다.

In [ ]:
with tf.variable_scope("relu", reuse=True):
    threshold = tf.get_variable("threshold")

밑과 같이 해도 된다.

In [ ]:
with tf.variable_scope("relu") as scope:
    scope.reuse_variables()
    threshold = tf.get_variable("threshold")

마지막으로 밑과 같이 사용하면 된다.

In [ ]:
def relu(X):
    with tf.variable_scope("relu", reuse=True):
        threshold = tf.get_variable("threshold") # reuse existing variable 
        [...]
        return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X") 
with tf.variable_scope("relu"): # create the variable 
    threshold = tf.get_variable("threshold", shape=(),
                                initializer=tf.constant_initializer(0.0))
    
relus = [relu(X) for relu_index in range(5)]
output = tf.add_n(relus, name="output")

<img src="ch9_10.png" alt="Drawing" style="width: 700px;"/>

위와 같이 threshold를 relu함수 안에 선언하고 싶다면 다음과 같이 하면 된다.

In [ ]:
def relu(X):
    threshold = tf.get_variable("threshold", shape=(),
                            initializer=tf.constant_initializer(0.0)) #reuse it
    [...] 
    return tf.maximum(z, threshold, name="max")

X = tf.placeholder(tf.float32, shape=(None, n_features), name="X") 
relus = [] 

for relu_index in range(5):
    with tf.variable_scope("relu", reuse=(relu_index >= 1)) as scope:
        relus.append(relu(X)) 
    output = tf.add_n(relus, name="output")

그렇다면 다음과 같은 그래프가 그려진다(첫번째 relu의 threshold 재사용)

<img src="ch9_11.png" alt="Drawing" style="width: 700px;"/>